# OMOP Create Dataset to compute sofa score

The Sequential Organ Failure Assessment (SOFA) is a morbidity severity score and mortality estimation tool developed from a large sample of ICU patients throughout the world. Unlike other scoring systems, such as the SAPS II and APACHE II systems, the SOFA was designed to focus on organ dysfunction and morbidity, with less of an emphasis on mortality prediction. The authors designed the system with an emphasis on bedside applicability and simplicity using widely available variables.

Connect to database

In [28]:
import __init__

In [39]:
import numpy as np
import pandas as pd
import datetime

# Open a connection to OMOP
from fleming_lib.tools import connect_to_omop
conn = connect_to_omop()

## Select list of patients

### SQL requests


- not be "limited" (Concept_id: 2001018843 and 2001030812 and 4127294)


In [30]:
query = """
select
    distinct person_id
from 
    measurement 
where 
    measurement_source_concept_id in (2001018843, 2001030812, 4127294)  
    ;"""

limitation = pd.read_sql_query(query, conn)

- Age >= 15 
- Processed only by intensive care unit

In [4]:
# in ICU
query = """
with icu as (
   select
       care_site_id, care_site_name, place_of_service_source_value
   from
       care_site
   where
       lower(place_of_service_source_value) like '%intensive%'
    or
        place_of_service_source_value = 'Cardiac surgery recovery unit' -- c'est aussi ICU
    or
        place_of_service_source_value = 'Coronary care unit'  -- c'est aussi ICU
   and
       lower(place_of_service_source_value) not like '%neonatal%'    -- removing Neonatal ICUs
   )
select
   distinct vd.person_id
from
   visit_detail vd
join
   icu
on
   icu.care_site_id = vd.care_site_id
;"""

icu = pd.read_sql_query(query, conn)

In [5]:
# age > 15
query = """
with person_age as (
    select
        p.person_id, (v.visit_start_date - p.birth_datetime)/365.25 age
    from
        person p
    left outer join
        visit_occurrence v
    on
        p.person_id = v.person_id)
select
    person_id, age
from
    person_age
where
    age > 15            -- including persons older than 15
and 
    age < 150             -- removing any outliers
;"""

age = pd.read_sql_query(query, conn)

### Cohort fusion

In [6]:
list_patient = list(set(icu.person_id) & set(age.person_id) & set(limitation.person_id))
list_patient[:10]

[62063367,
 62063368,
 62063369,
 62063370,
 62063371,
 62063372,
 62063373,
 62063374,
 62063377,
 62063378]

In [7]:
len(list_patient)

31056

## Build dataset for each patient

### Extract measurement

#### SQL requests

In [40]:
patient_id = (62063368, 62063378)  # person_id

In [46]:
query = """
select
    distinct m.person_id, m.measurement_datetime,measurement_concept_id, m.measurement_concept_name, m.value_source_value, m.unit_source_value, d.death_datetime
from 
    measurement m 
left join 
    death d on d.person_id = m.person_id
where
    m.measurement_concept_id in
    (3024929,   -- platelets
     4233883,   -- pa02/FiO2
     3024128,   -- bilirubin
     3027598,   -- map_bp
     3016723,   -- creatinine
     3032652    -- glasgow coma scale
    )
and m.person_id in {}
order by m.person_id, m.measurement_datetime
;""".format(patient_id)

measures = pd.read_sql_query(query, conn)
measures.iloc[:10]

,person_id,measurement_datetime,measurement_concept_id,measurement_concept_name,value_source_value,unit_source_value,death_datetime
0,62063368,2188-11-12 07:15:00,3016723,Creatinine serum/plasma,0.5,mg/dL,2188-11-22 12:00:00
1,62063368,2188-11-12 07:15:00,3024929,Platelets [#/volume] in Blood by Automated count,720,K/uL,2188-11-22 12:00:00
2,62063368,2188-11-12 08:04:00,3024128,Total Bilirubin serum/plasma,0.3,mg/dL,2188-11-22 12:00:00
3,62063368,2188-11-12 10:00:00,3032652,Glasgow coma scale,15,points,2188-11-22 12:00:00
4,62063368,2188-11-12 10:00:00,3027598,Mean blood pressure,107.33300018310547,mmHg,2188-11-22 12:00:00
5,62063368,2188-11-12 10:30:00,3027598,Mean blood pressure,110,mmHg,2188-11-22 12:00:00
6,62063368,2188-11-12 10:45:00,3027598,Mean blood pressure,112.66699981689453,mmHg,2188-11-22 12:00:00
7,62063368,2188-11-12 11:00:00,3027598,Mean blood pressure,107.33300018310547,mmHg,2188-11-22 12:00:00
8,62063368,2188-11-12 12:00:00,3027598,Mean blood pressure,111.66699981689453,mmHg,2188-11-22 12:00:00
9,62063368,2188-11-12 13:00:00,3027598,Mean blood pressure,124.66699981689453,mmHg,2188-11-22 12:00:00


#### Change type datetime

In [15]:
measures['death_datetime'] = pd.to_datetime(measures['death_datetime'])
measures['measurement_datetime'] = pd.to_datetime(measures['measurement_datetime'])
measures.iloc[:10]

,person_id,measurement_datetime,measurement_concept_id,measurement_concept_name,value_source_value,unit_source_value,death_datetime
0,62063368,2188-11-12 07:15:00,3016723,Creatinine serum/plasma,5.000,mg/dL,2188-11-22 12:00:00
1,62063368,2188-11-12 07:15:00,3024929,Platelets [#/volume] in Blood by Automated count,720.000,K/uL,2188-11-22 12:00:00
2,62063368,2188-11-12 08:04:00,3024128,Total Bilirubin serum/plasma,5.130,mg/dL,2188-11-22 12:00:00
3,62063368,2188-11-12 10:00:00,3032652,Glasgow coma scale,15.000,points,2188-11-22 12:00:00
4,62063368,2188-11-12 10:00:00,3027598,Mean blood pressure,107.333,mmHg,2188-11-22 12:00:00
5,62063368,2188-11-12 10:30:00,3027598,Mean blood pressure,110.000,mmHg,2188-11-22 12:00:00
6,62063368,2188-11-12 10:45:00,3027598,Mean blood pressure,112.667,mmHg,2188-11-22 12:00:00
7,62063368,2188-11-12 11:00:00,3027598,Mean blood pressure,107.333,mmHg,2188-11-22 12:00:00
8,62063368,2188-11-12 12:00:00,3027598,Mean blood pressure,111.667,mmHg,2188-11-22 12:00:00
9,62063368,2188-11-12 13:00:00,3027598,Mean blood pressure,124.667,mmHg,2188-11-22 12:00:00


#### Homogenise units

In [16]:
from fleming_lib.utils import convert_frac, to_numeric
measures = convert_frac(measures, "value_source_value")
measures = to_numeric(measures, "value_source_value")

In [17]:
#creatinine : convert to mg/L
measures.loc[measures["measurement_concept_name"]=="Creatinine serum/plasma","value_source_value"]*=10

In [18]:
#bilurin : convert to umol/L
measures.loc[measures["measurement_concept_name"]=="Total Bilirubin serum/plasma","value_source_value"]*=17.1

#### Pivot table

In [19]:
measures_piv = measures.pivot_table(index=['measurement_datetime','person_id'], columns='measurement_concept_name', values='value_source_value', aggfunc='first')
measures_piv.reset_index(inplace=True)
measures_piv.columns.name = None
measures_piv.iloc[:10]

,measurement_datetime,person_id,Creatinine serum/plasma,Glasgow coma scale,Mean blood pressure,Platelets [#/volume] in Blood by Automated count,Total Bilirubin serum/plasma
0,2153-09-20 14:50:00,62063378,130.0,NaN,NaN,138.0,NaN
1,2153-09-25 18:00:00,62063378,NaN,15.0,NaN,NaN,NaN
2,2153-09-25 19:22:00,62063378,100.0,NaN,NaN,107.0,NaN
3,2153-09-25 20:00:00,62063378,NaN,NaN,100.0,NaN,NaN
4,2153-09-25 21:00:00,62063378,NaN,NaN,94.0,NaN,NaN
5,2153-09-25 22:00:00,62063378,NaN,15.0,107.0,NaN,NaN
6,2153-09-25 23:00:00,62063378,NaN,NaN,99.0,NaN,NaN
7,2153-09-26 00:00:00,62063378,NaN,15.0,104.0,NaN,NaN
8,2153-09-26 01:00:00,62063378,NaN,NaN,104.0,NaN,NaN
9,2153-09-26 02:00:00,62063378,NaN,NaN,76.0,NaN,NaN


### Extract drugs exposure

#### SQL requests

In [32]:
# Get data

query = """
select
    distinct person_id, drug_exposure_start_datetime,drug_concept_id,drug_source_value,quantity_source_value,dose_unit_source_value
from 
    drug_exposure

where
    drug_concept_id in
    (1337860,   -- dopamine
     1343916,   -- epinephrine
     1344056,   -- epinephrine
     1321341,   -- norepinephrine
     1321364,   -- norepinephrine
     1337720   -- dobutamine
    )
  
    
and person_id in {}
order by person_id, drug_exposure_start_datetime
;""".format(patient_id)

drugs = pd.read_sql_query(query, conn)
drugs.iloc[:10]

,person_id,drug_exposure_start_datetime,drug_concept_id,drug_source_value,quantity_source_value,dose_unit_source_value
0,62063368,NaT,1337720,Dobutamine,0,mg
1,62063368,NaT,1337720,Dobutamine,2.9951999187,mg
2,62063368,NaT,1337720,Dobutamine,7.0655999184,mg
3,62063368,NaT,1337720,Dobutamine,7.6799998283,mg
4,62063368,NaT,1337720,Dobutamine,1.2666666508,mg
5,62063368,NaT,1337720,Dobutamine,15.3599996567,mg
6,62063368,NaT,1337720,Dobutamine,7.9159998894,mg
7,62063368,2188-11-13 08:50:00,1337720,Dobutamine,2.4739582539,mcg/kg/min
8,62063368,2188-11-13 09:15:00,1337720,Dobutamine,5,mcg/kg/min
9,62063368,2188-11-13 09:17:00,1337720,Dobutamine,2.5,mcg/kg/min


In [33]:
drugs=drugs[drugs['drug_exposure_start_datetime'].isnull()==0]
drugs.iloc[:10]

,person_id,drug_exposure_start_datetime,drug_concept_id,drug_source_value,quantity_source_value,dose_unit_source_value
7,62063368,2188-11-13 08:50:00,1337720,Dobutamine,2.4739582539,mcg/kg/min
8,62063368,2188-11-13 09:15:00,1337720,Dobutamine,5,mcg/kg/min
9,62063368,2188-11-13 09:17:00,1337720,Dobutamine,2.5,mcg/kg/min
10,62063368,2188-11-13 09:25:00,1337720,Dobutamine,2.5,mcg/kg/min
11,62063368,2188-11-13 09:35:00,1337720,Dobutamine,2.5,mcg/kg/min
12,62063368,2188-11-13 09:40:00,1337720,Dobutamine,2.5,mcg/kg/min
13,62063368,2188-11-13 09:45:00,1337720,Dobutamine,2.5,mcg/kg/min
14,62063368,2188-11-13 09:50:00,1337720,Dobutamine,2.5,mcg/kg/min
15,62063368,2188-11-13 09:55:00,1337720,Dobutamine,2.5,mcg/kg/min
16,62063368,2188-11-13 10:00:00,1337720,Dobutamine,2.5,mcg/kg/min


In [37]:
measures.columns

Index(['person_id', 'measurement_datetime', 'measurement_concept_id',
       'measurement_concept_name', 'value_source_value', 'unit_source_value',
       'death_datetime'],
      dtype='object')

#### Standardize drugs exposure

In [34]:
drugs['death_datetime']=pd.datetime(2012, 5, 1)
drugs.columns=measures.columns
drugs.iloc[:10]

,person_id,measurement_datetime,measurement_concept_id,measurement_concept_name,value_source_value,unit_source_value,death_datetime
7,62063368,2188-11-13 08:50:00,1337720,Dobutamine,2.4739582539,mcg/kg/min,2012-05-01
8,62063368,2188-11-13 09:15:00,1337720,Dobutamine,5,mcg/kg/min,2012-05-01
9,62063368,2188-11-13 09:17:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
10,62063368,2188-11-13 09:25:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
11,62063368,2188-11-13 09:35:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
12,62063368,2188-11-13 09:40:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
13,62063368,2188-11-13 09:45:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
14,62063368,2188-11-13 09:50:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
15,62063368,2188-11-13 09:55:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
16,62063368,2188-11-13 10:00:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01


In [35]:
drugs['measurement_datetime']=drugs['measurement_datetime']- pd.Timedelta('1 days')

In [36]:
drugs.head()

,person_id,measurement_datetime,measurement_concept_id,measurement_concept_name,value_source_value,unit_source_value,death_datetime
7,62063368,2188-11-12 08:50:00,1337720,Dobutamine,2.4739582539,mcg/kg/min,2012-05-01
8,62063368,2188-11-12 09:15:00,1337720,Dobutamine,5,mcg/kg/min,2012-05-01
9,62063368,2188-11-12 09:17:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
10,62063368,2188-11-12 09:25:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01
11,62063368,2188-11-12 09:35:00,1337720,Dobutamine,2.5,mcg/kg/min,2012-05-01


In [19]:
for pid in patient_id:
    deathtime=measures[measures['person_id']==pid]['death_datetime']
    if sum(deathtime.isnull()) :
        deathtime=0
    else :
        deathtime=deathtime[0]
    drugs.loc[drugs['person_id']==pid,'death_datetime']=deathtime
    
drugs.head()

,person_id,measurement_datetime,measurement_concept_id,measurement_concept_name,value_source_value,unit_source_value,death_datetime
7,62063368,2188-11-12 08:50:00,1337720,Dobutamine,2.4739582539,mcg/kg/min,2188-11-22 12:00:00
8,62063368,2188-11-12 09:15:00,1337720,Dobutamine,5,mcg/kg/min,2188-11-22 12:00:00
9,62063368,2188-11-12 09:17:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
10,62063368,2188-11-12 09:25:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
11,62063368,2188-11-12 09:35:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00


#### Change type datetime

In [20]:
drugs['death_datetime'] = pd.to_datetime(drugs['death_datetime'])
drugs['measurement_datetime'] = pd.to_datetime(drugs['measurement_datetime'])
drugs.iloc[:10]

,person_id,measurement_datetime,measurement_concept_id,measurement_concept_name,value_source_value,unit_source_value,death_datetime
7,62063368,2188-11-12 08:50:00,1337720,Dobutamine,2.4739582539,mcg/kg/min,2188-11-22 12:00:00
8,62063368,2188-11-12 09:15:00,1337720,Dobutamine,5,mcg/kg/min,2188-11-22 12:00:00
9,62063368,2188-11-12 09:17:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
10,62063368,2188-11-12 09:25:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
11,62063368,2188-11-12 09:35:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
12,62063368,2188-11-12 09:40:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
13,62063368,2188-11-12 09:45:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
14,62063368,2188-11-12 09:50:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
15,62063368,2188-11-12 09:55:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00
16,62063368,2188-11-12 10:00:00,1337720,Dobutamine,2.5,mcg/kg/min,2188-11-22 12:00:00


#### Pivot Table

In [21]:
drugs_piv = drugs.pivot_table(index=['measurement_datetime','person_id'], columns='measurement_concept_name', values='value_source_value', aggfunc='first')
drugs_piv.reset_index(inplace=True)
drugs_piv.columns.name = None
drugs_piv.iloc[:10]

,measurement_datetime,person_id,Dobutamine
0,2188-11-12 08:50:00,62063368,2.4739582539
1,2188-11-12 09:15:00,62063368,5
2,2188-11-12 09:17:00,62063368,2.5
3,2188-11-12 09:25:00,62063368,2.5
4,2188-11-12 09:35:00,62063368,2.5
5,2188-11-12 09:40:00,62063368,2.5
6,2188-11-12 09:45:00,62063368,2.5
7,2188-11-12 09:50:00,62063368,2.5
8,2188-11-12 09:55:00,62063368,2.5
9,2188-11-12 10:00:00,62063368,2.5


In [22]:
col=drugs_piv.columns
numerical_variables = col[2:]

drugs_piv = convert_frac(drugs_piv, numerical_variables)
drugs_piv = to_numeric(drugs_piv, numerical_variables)

### Merge both dataset

In [23]:
data = pd.merge(measures_piv, drugs_piv, on='person_id')
data.head()

,measurement_datetime_x,person_id,Creatinine serum/plasma,Glasgow coma scale,Mean blood pressure,Platelets [#/volume] in Blood by Automated count,Total Bilirubin serum/plasma,measurement_datetime_y,Dobutamine
0,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2188-11-12 08:50:00,2.473958
1,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2188-11-12 09:15:00,5.000000
2,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2188-11-12 09:17:00,2.500000
3,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2188-11-12 09:25:00,2.500000
4,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2188-11-12 09:35:00,2.500000


In [24]:
col=data.columns
del data["measurement_datetime_y"]
data=data.rename(index=str, columns={"measurement_datetime_x": "measurement_datetime"})
data.head()

,measurement_datetime,person_id,Creatinine serum/plasma,Glasgow coma scale,Mean blood pressure,Platelets [#/volume] in Blood by Automated count,Total Bilirubin serum/plasma,Dobutamine
0,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2.473958
1,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,5.000000
2,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2.500000
3,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2.500000
4,2188-11-12 07:15:00,62063368,5.0,NaN,NaN,720.0,NaN,2.500000


In [25]:
df=data
variable_names = df.columns.values
mat = np.asarray(df)

### Fill missing values

In [26]:
from datetime import timedelta
from fleming_lib.preprocessing import fill_last_upto

# Fill missing values with last one (up to h time prior to measurement)
df = df.groupby('person_id', group_keys=False).apply(fill_last_upto, h=timedelta(hours=24))



In [27]:
df=df.sort_values(by=['measurement_datetime'])
df.head()


,measurement_datetime,person_id,Creatinine serum/plasma,Glasgow coma scale,Mean blood pressure,Platelets [#/volume] in Blood by Automated count,Total Bilirubin serum/plasma,Dobutamine
0,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.473958
22,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000
23,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000
24,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000
25,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000


## Compute SOFA score

In [28]:
def compute_sofa_score(row):
    """
    Compute the SOFA score

    Args:
        #measurement                concept_id
        paO2/FiO2 (float):          4233883 
        platelets (float):          3024929
        bilirubin (float):          3024128
        map (float):                3027598         
        glasgow-coma_score (float): 3032652
        creatinine (float):         3016723 

        #drugs
        dopamine (float):           1337860
        epinephrine (float):        1343916 / 1344056
        norepinephrine (float):     1321341 / 1321364
        dobutamine (float):         1337720

    Returns:
        sofa_score (int): SOFA score 
    """
   
    sofa_score = 0 

    if 'Oxygen saturation in Arterial blood' in row:

        paO2FiO2= row['Oxygen saturation in Arterial blood']
    
        if(paO2FiO2<100):
            sofa_score+=4 
        elif(paO2FiO2<200):
            sofa_score+=3
        elif(paO2FiO2<300):
            sofa_score+=2
        elif(paO2FiO2<400):
            sofa_score+=1

    if 'Platelets [#/volume] in Blood by Automated count' in row:

        platelets=row['Platelets [#/volume] in Blood by Automated count']

        if(platelets<20):
            sofa_score+=4 
        elif(platelets<50):
            sofa_score+=3
        elif(platelets<100):
            sofa_score+=2
        elif(platelets<150):
            sofa_score+=1  

    if 'Total Bilirubin serum/plasma' in row:

        bilirubin=row['Total Bilirubin serum/plasma']
        
        if(bilirubin>204):
            sofa_score+=4 
        elif(bilirubin>=102):
            sofa_score+=3
        elif(bilirubin>=33):
            sofa_score+=2
        elif(bilirubin>=20):
            sofa_score+=1
            

    if 'Dopamine' in row:

        dopamine=row['Dopamine']      

        if(dopamine>15):
            sofa_score+=4 
        elif(dopamine>5):
            sofa_score+=3
        elif(dopamine<=5):
            sofa_score+=2
    elif 'Norepinephrine' in row:

        norepinephrine=row['Norepinephrine']

        if(norepinephrine>0.1):
            sofa_score+=4
        else:
            sofa_score+=3

    elif 'Epinephrine' in row:

        epinephrine=row['Epinephrine']

        if(epinephrine>0.1):
            sofa_score+=4
        else:
            sofa_score+=3

    elif 'Dobutamine' in row:

        dobutamine=row['Dobutamine']

        if(dobutamine>0):
            sofa_score+=2

    elif 'Mean blood pressure' in row:

        mbp=row['Mean blood pressure']

        if(mbp<70):
            sofa_score+=1

    if 'Glasgow coma scale' in row:

        glasgow_coma_score=row['Glasgow coma scale']    

        if(glasgow_coma_score<6):
            sofa_score+=4 
        elif(glasgow_coma_score>=6 and glasgow_coma_score<=9):
            sofa_score+=3
        elif(glasgow_coma_score>=10 and glasgow_coma_score<=12):
            sofa_score+=2
        elif(glasgow_coma_score>=13 and glasgow_coma_score<=14):
            sofa_score+=1
    

    if 'Creatinine serum/plasma' in row:

        creatinine=row['Creatinine serum/plasma']    

        if(creatinine>50):
            sofa_score+=4 
        elif(creatinine>=35 and creatinine <=49):
            sofa_score+=3
        elif(creatinine>=20 and creatinine <=34):
            sofa_score+=2
        elif(creatinine>=12 and creatinine <=19):
            sofa_score+=1

    

    return sofa_score

In [29]:
tempRelevantComponents = df
#relevantComponents.dropna(inplace=True)
tempRelevantComponentsScore = tempRelevantComponents.apply(compute_sofa_score, axis=1)
df['sofa score']=tempRelevantComponentsScore
df.head(10)

,measurement_datetime,person_id,Creatinine serum/plasma,Glasgow coma scale,Mean blood pressure,Platelets [#/volume] in Blood by Automated count,Total Bilirubin serum/plasma,Dobutamine,sofa score
0,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.473958,2
22,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
23,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
24,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
25,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
26,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
27,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
29,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
21,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2
30,2188-11-12 07:15:00,62063368,5.0,NaT,NaT,720.0,NaT,2.500000,2


In [30]:
df.tail(10)

,measurement_datetime,person_id,Creatinine serum/plasma,Glasgow coma scale,Mean blood pressure,Platelets [#/volume] in Blood by Automated count,Total Bilirubin serum/plasma,Dobutamine,sofa score
19898,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19900,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19901,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19902,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19903,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19904,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19905,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19906,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19908,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,2.5,9
19928,2188-11-22 19:00:00,62063368,7.0,3,26,125.0,34.2,1.3,9
